# Notebook for Lab 1 - Symmetric Encryption
By Luis Daniel Casais Mezquida  
Cryptography 22/23  
Bachelor's degree in Applied Mathematics and Computing, grp. 121  
Universidad Carlos III de Madrid

## General goal
In this lab you’ll get familiarized with programming (*using*) symmetric encryption algorithms using a modern library. The lab makes use of Python language but it could also be done with Java or C-like languages. 

## Specific goals
- Make use of different data representation formats (byte/ascii, string, hexadecimal, b64). Students should be able to recognize each of the formats and translate from one format to another.
- Make use of the pyca/cryptography library to encrypt and decrypt simple messages (strings) with block ciphers (AES, Camellia) operated under different modes (ECB, CBC and CTR) and considering padding issues.
- Make use of a robust pseudo random number generator function/algorithm to generate keys, initialization vectors and nonces.
- Understand how modifications in the ciphertext affect the decryption result when different operation modes are used.
- Make use of the pyca/cryptography library to encrypt and decrypt simple messages with stream ciphers (ChaCha20).
- Make use of the pylfsr library to generate a keystream from a LFSR.

## Modules
Cryptography library includes both high level recipes and low-level interfaces to common cryptographic algorithms such as symmetric ciphers, message digests, and key derivation functions. For more details you can consult [the following link](https://pypi.org/project/cryptography).  
For the moment, we are going to directly use the primitives, forgetting about Fernet constructions. You’ll may find useful to have these links as reference:
- [Symmetric encryption](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/)
- [Symmetric padding](https://cryptography.io/en/latest/hazmat/primitives/padding/)
- [Link to info about os library](https://docs.python.org/3/library/os.html)
- [Link to info about base64 library](https://docs.python.org/3/library/base64.html)
- [Link info about pylfsr library](https://pypi.org/project/pylfsr/)
- [Link to your very first encryption/decryption code snippet example](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.Cipher)

## Lab assignment and assessment
You are given one Python script/notebook, that contains the headers of all the functions you should develop at the end of this lab. Next, you’ll be guided through several tasks you’ll have to complete in order to do so.

Lab 1 assessment will be performed with a test/quiz/exam at the end of the course (along the other lab assessments).

# 0. Modules installation and imports

In [ ]:
!pip install cryptography
!pip install pylfsr

In [2]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

from cryptography.hazmat.primitives import padding

from pylfsr import LFSR

import base64

import os 

import secrets

# 1. Symmetric Block Ciphers

Next, you’ll learn how to encrypt a simple text with symmetric block ciphers. First, you’ll use AES cipher operated under several confidentiality operation modes, and then you’ll be prompted to use other symmetric ciphers (CAMELLIA).
Read AND USE AS REFERENCE the information provided in the documentation web page of the pyca/cryptography library for symmetric encryption [here](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/).

## 1.1 BYTE-LIKE DATA

### Task 1.1.1 From byte-like to hex and b64

First, let’s see a primer in dealing with binary data in Python, as it is a very frequent data format when using cryptography. The code lines for defining the key, the initialization vector and the text that you’ll use in your first encryption are given next:

In [3]:
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
iv = b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'
message = b'a secret message'

Notice the preceding “b” in the previous data in single quotes. It is used to transform the provided (“text”-like) data into binary format (following [ASCII codification](https://en.wikipedia.org/wiki/ASCII)). That is, the code will treat these data as binary. Each byte can be specified by:
- the ASCII character corresponding to that byte if it is printable, 
- the hexadecimal representation of the byte, preceding the 2-hex digits by “\x”.

Some examples: 
- b’MN’ corresponds to binary data 4d4e <sub>(16</sub> = 0100 1101 0100 1110 <sub>(2</sub> 
- b’\xa1\xf7’ corresponds to binary data a1f7 <sub>(16</sub> = 1010 0001 1111 0111 <sub>(2</sub>

You can get the hexadecimal representation of binary data using the built-in Python function <code>hex()</code> as follows:

_Note: <code>hex()</code> belongs to the set of [Python built-in functions](https://docs.python.org/3/library/functions.html)._

In [4]:
binarydata = b'MN\\\xde\xa1' 
hex_binarydata = binarydata.hex() 
print("hex_binarydata =", hex_binarydata)

hex_binarydata = 4d4e5cdea1


You can get back your bytes from their hex representation as follows using [<code>bytes.fromhex()</code>](https://docs.python.org/3/library/stdtypes.html?highlight=fromhex#bytes.fromhex):

In [5]:
retrieved_binarydata = bytes.fromhex(hex_binarydata) 
print("retrieved_binarydata =", retrieved_binarydata)

retrieved_binarydata = b'MN\\\xde\xa1'


Other codification that can be used to deal in a print-friendly manner with binary data is [B64](https://en.wikipedia.org/wiki/Base64). 

An option is to use the Python module [base64](https://docs.python.org/3/library/base64.html). In order to use it you have to import the module in your code (you should have done it in section 0) and use <code>base64.b64encode()</code> as follows:

In [6]:
# import base64 

b64_binarydata = base64.b64encode(binarydata)
print("b64_binarydata =",b64_binarydata)

b64_binarydata = b'TU5c3qE='


As before, you can get your bytes back from B64 as follows using <code>base64.b64decode()</code>:

In [7]:
retrieved_binarydata = base64.b64decode(b64_binarydata)
print("retrieved_binarydata =", retrieved_binarydata)

retrieved_binarydata = b'MN\\\xde\xa1'


Represent the given key, IV and message text in hexadecimal and B64 and check that you understand how binary data is provided and represented. Undo the hex and B64 representations by retrieving back the bytes and print the results.

In [8]:
key_hex = key.hex()
key_b64 = base64.b64encode(key)

print("key_hex =", key_hex)
print("key_b64 =", key_b64)

key_hex_dec = bytes.fromhex(key_hex)
key_b64_dec = base64.b64decode(key_b64)

print("key_hex_dec =", key_hex_dec)
print("key_b64_dec =", key_b64_dec)

key_hex = 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a
key_b64 = b'TU5c3qH3nM8QQageAA5lceg2HBIYphD7uV4KLDCia2o='
key_hex_dec = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
key_b64_dec = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'


## 1.2 AES256 - CBC MODE

### Task 1.2.1 Encrypt/decrypt with AES 256 CBC

Now, let’s encrypt a simple text (message) using the AES cipher with a 256-bit key and operating the cipher under CBC mode.

Read the information and take the code provided as example [here](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.Cipher). 

Notice that Cipher is a general construction that can be instantiated with a specific algorithm, mode and backend.

_Note: We will rely on the default backend but if you want to know more about backends, read [this web page](https://cryptography.io/en/latest/hazmat/backends/)._

In this case we’ll use the classes <code>algorithms.AES(..)</code> and <code>modes.CBC(..)</code> for the first two parameters.

Notice also that in the example [in this web page](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.Cipher) the key and iv data are randomly obtained.

Copy and change the code of the example (given at the web page of the library) so you use the key, iv and message given in Section 1.1. Leave the rest of the code unchanged. Additionally, insert calls to the <code>print()</code> function so the message, key, iv and the results of the encryption and decryption processes are printed.

In [15]:
## encryption code
cipher = Cipher(algorithms.AES256(key), mode = modes.CBC(iv))
encryptor = cipher.encryptor()
ct = encryptor.update(message) + encryptor.finalize()

print("ciphertext =", ct)

## decryption code
decryptor = cipher.decryptor()
pt = decryptor.update(ct) + decryptor.finalize()

print("plaintext =", pt)

ciphertext = b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeN'
plaintext = b'a secret message'


Check your results with the following values:
```
message = b'a secret message'
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
ciphertext = b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeN'
hex_ciphertext = 3ba6e5853b5ec226ec077c03cc0dae4e
b64_ciphertext = b'O6blhTtewibsB3wDzA2uTg=='
plaintext = b'a secret message'
hex_plaintext = 6120736563726574206d657373616765
b64_plaintext = b'YSBzZWNyZXQgbWVzc2FnZQ=='
```

Which length in bytes has the input data to encrypt in Task 1.2.2? And the output ciphertext?

Notice that you may use the built-in function [<code>len()</code>](https://docs.python.org/3/library/functions.html#len) to get the length of an object.

In [17]:
print(len(message))
print(len(ct))

16
16



### Task 1.2.2 Adding padding to AES 256 CBC

Now let’s play with the length of the message to encrypt! In the previous exercise you encrypted a message that has an exact length of 16 characters, that is, the AES block size. Try to encrypt a longer message whose length in characters is not multiple of 16. You’ll get an error for sure! It is because if you need to encrypt a message with length not multiple of the cipher block size and you are using CBC operation mode, you need to add padding to your input so its length satisfies that condition.

Read the information provided in the documentation web page of the pyca/cryptography library on how the library deals with symmetric padding [here](https://cryptography.io/en/latest/hazmat/primitives/padding/).

Study the code in the example provided to illustrate the PKCS7 padding [here](https://cryptography.io/en/latest/hazmat/primitives/padding/#cryptography.hazmat.primitives.padding.PKCS7).

Notice that you have to import the padding module from the pyca/cryptography library (if you have not done it in Section 0). Notice also that the correct order to combine padding-encryption and decryption-unpadding is the following one:
1. Pad the data to encrypt to obtain the padded data
2. Encrypt the padded data to obtain the encrypted text
3. --- (simulation of data exchange) ----
4. Decrypt the encrypted text to obtain the padded plaintext
5. Unpad the padded plaintext to obtain the original plaintext

After studying it, incorporate PKCS7 padding in your previous code.

In [37]:
block_size = 16*8  # bits
message = b'a secret message'

# pad
padder = padding.PKCS7(block_size).padder()
padded_message = padder.update(message) + padder.finalize()
print("padded_message =", padded_message, "len =", len(padded_message))

# encrypt
cipher = Cipher(algorithms.AES256(key), mode = modes.CBC(iv))
encryptor = cipher.encryptor()
ct = encryptor.update(padded_message) + encryptor.finalize()

print("ciphertext =", ct)

## decrypt
decryptor = cipher.decryptor()
padded_pt = decryptor.update(ct) + decryptor.finalize()

print("padded_pt =", padded_pt)

unpadder = padding.PKCS7(block_size).unpadder()
pt = unpadder.update(padded_pt) + unpadder.finalize()

print("plaintext =", pt)


padded_message = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10' len = 32
ciphertext = b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeNq\xb6\xb8\xb5\x06\xeeJ\xd9\xfb\x004\xb5\xac\x9e}\xf0'
padded_pt = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'
plaintext = b'a secret message'


Your previous AES256-CBC code, updated to include PKCS7 padding, with the data provided in Section 1.1 (key, iv and message), should provide the result shown next:
```
message = b'a secret message'
padded_message = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10' length =  32
ciphertext = b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeNq\xb6\xb8\xb5\x06\xeeJ\xd9\xfb\x004\xb5\xac\x9e}\xf0'
hex_ciphertext = 3ba6e5853b5ec226ec077c03cc0dae4e71b6b8b506ee4ad9fb0034b5ac9e7df0
padded_plaintext = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'
hex_padded_plaintext = 6120736563726574206d65737361676510101010101010101010101010101010
plaintext = b'a secret message'
hex_plaintext = 6120736563726574206d657373616765
```

Answer the following questions:
- Which length in bytes has the input data to encrypt in Task 1.2.2?
- Which length in bytes has the output ciphertext in Task 1.2.2?
- How many bytes have been added as PKCS7 padding?
- Which value has been used in the bytes added as padding?

In [38]:
print(len(message))
print(len(ct))
print(len(padded_message) - len(message))
print(padded_message[-1])

16
32
16
16


### Task 1.2.3 Checking AES 256 CBC PKCS7

Check that your code works correctly by encrypting <code>other_message</code>, a message with 25 characters (length not multiple of 16) and provided next, and the same <code>key</code> and <code>iv</code> already used.

In [51]:
block_size = 16*8  # bits
other_message = b'This message is important'

# pad
padder = padding.PKCS7(block_size).padder()
padded_message = padder.update(other_message) + padder.finalize()
print("padded_message =", padded_message, "len =", len(padded_message))

# encrypt
cipher = Cipher(algorithms.AES256(key), mode = modes.CBC(iv))
encryptor = cipher.encryptor()
ct = encryptor.update(padded_message) + encryptor.finalize()

print("ciphertext =", ct)

# decrypt
decryptor = cipher.decryptor()
padded_pt = decryptor.update(ct) + decryptor.finalize()

print("padded_pt =", padded_pt)

# unpad
unpadder = padding.PKCS7(block_size).unpadder()
pt = unpadder.update(padded_pt) + unpadder.finalize()

print("plaintext =", pt)

padded_message = b'This message is important\x07\x07\x07\x07\x07\x07\x07' len = 32
ciphertext = b'\x89\x00g\x1aP4\xf7W\x16\xc1\xffi\xbb\xc8\x9f\xcf\xd3\x84D\x1d\nL\xcbvb\xf1^sn\xfa\x06Z'
padded_pt = b'This message is important\x07\x07\x07\x07\x07\x07\x07'
plaintext = b'This message is important'


You should get the following results:
```
other_message = b'This message is important'
padded_other_message = b'This message is important\x07\x07\x07\x07\x07\x07\x07'
length =  32
ciphertext = b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeNq\xb6\xb8\xb5\x06\xeeJ\xd9\xfb\x004\xb5\xac\x9e}\xf0'
hex_ciphertext = 3ba6e5853b5ec226ec077c03cc0dae4e71b6b8b506ee4ad9fb0034b5ac9e7df0
padded_plaintext = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'
hex_padded_plaintext = 6120736563726574206d65737361676510101010101010101010101010101010
plaintext = b'a secret message'
hex_plaintext = 6120736563726574206d657373616765
```

Answer the following questions:
- Which length in bytes has the input data to encrypt in Task 1.2.3?
- Which length in bytes has the output ciphertext in Task 1.2.3?
- How many bytes have been added as PKCS7 padding?
- Which value has been used in the bytes added as padding?

In [52]:
print(len(other_message))
print(len(ct))
print(len(padded_message) - len(other_message))
print(padded_message[-1])

25
32
7
7


### Task 1.2.4 Wrapping AES 256 CBC PKCS7 encryption/decryption in functions

Let’s wrap your AES256-CBC-PKCS7 code in two functions, one for encryption and another one for decryption. Next you have the headers that you should use for these two functions.

In [63]:
def aes256_cbc_pkcs7_encrypt(data_to_encrypt, key, iv):
    BLOCK_SIZE = 16*8  # bits
    
    # pad
    padder = padding.PKCS7(BLOCK_SIZE).padder()
    padded_message = padder.update(data_to_encrypt) + padder.finalize()

    # encrypt
    cipher = Cipher(algorithms.AES256(key), mode = modes.CBC(iv))
    encryptor = cipher.encryptor()
    ct = encryptor.update(padded_message) + encryptor.finalize()
    
    return ct

def aes256_cbc_pkcs7_decrypt(encrypted_data, key, iv):
    BLOCK_SIZE = 16*8  # bits
    
    # decrypt
    cipher = Cipher(algorithms.AES256(key), mode = modes.CBC(iv))
    decryptor = cipher.decryptor()
    padded_pt = decryptor.update(encrypted_data) + decryptor.finalize()

    # unpad
    unpadder = padding.PKCS7(BLOCK_SIZE).unpadder()
    pt = unpadder.update(padded_pt) + unpadder.finalize()
    
    return pt

Test your functions with the data and results of Task 1.2.3

In [53]:
ct = aes256_cbc_pkcs7_encrypt(message, key, iv)
print("ct =", ct)

pt = aes256_cbc_pkcs7_decrypt(ct, key, iv)
print("pt =", pt)

ct = b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeNq\xb6\xb8\xb5\x06\xeeJ\xd9\xfb\x004\xb5\xac\x9e}\xf0'
pt = b'a secret message'


### Task 1.2.5 Checking AES 256 CBC PKCS7 against CyberChef

Test again your AES256-CBC-PKCS7 function but this time check your results using the web page tool [CyberChef](https://gchq.github.io/CyberChef/).  
You have to select, in the left sidebar, 'AES Encrypt' from 'Encryption / Encoding' and CBC mode.

## 1.3 AES256 - ECB MODE

Let’s now use the ECB operation mode. It is the simplest of all the confidentiality operation modes and does not need any parameter. You can consult the library documentation for this mode [here](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.modes.ECB).

### Task 1.3.1 Encrypt/decrypt with AES 256 ECB PKCS7

Write a new encryption and decryption code based on your AES256-CBC-PKCS7 code but that makes use of the ECB mode instead of the CBC one. Wrap the code in two functions (headers given below).

In [61]:
def aes256_ecb_pkcs7_encrypt(data_to_encrypt, key):
    BLOCK_SIZE = 16*8  # bits
    
    # pad
    padder = padding.PKCS7(BLOCK_SIZE).padder()
    padded_message = padder.update(data_to_encrypt) + padder.finalize()

    # encrypt
    cipher = Cipher(algorithms.AES256(key), mode = modes.ECB())
    encryptor = cipher.encryptor()
    ct = encryptor.update(padded_message) + encryptor.finalize()
    
    return ct

def aes256_ecb_pkcs7_decrypt(encrypted_data, key):
    BLOCK_SIZE = 16*8  # bits
    
    # decrypt
    cipher = Cipher(algorithms.AES256(key), mode = modes.ECB())
    decryptor = cipher.decryptor()
    padded_pt = decryptor.update(encrypted_data) + decryptor.finalize()

    # unpad
    unpadder = padding.PKCS7(BLOCK_SIZE).unpadder()
    pt = unpadder.update(padded_pt) + unpadder.finalize()
    
    return pt

In [62]:
ct = aes256_ecb_pkcs7_encrypt(message, key)
print("ct =", ct)

pt = aes256_ecb_pkcs7_decrypt(ct, key)
print("pt =", pt)

ct = b'i\xb0a\x1b\x95\x8f\x16\xd6\xed1\xc6\xfc\xa5\xab\xd5F|\x98\xfa\x0b\x8e\xab\xd4\xd7\x8e\x8f\xf5\n!&\xfd\xbb'
pt = b'a secret message'


You should obtain the following results if you encrypt <code>message</code> with the same <code>key</code> previously used.
```
message = b'a secret message'
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
padded_message = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10' 
length =  32
ciphertext = b'i\xb0a\x1b\x95\x8f\x16\xd6\xed1\xc6\xfc\xa5\xab\xd5F|\x98\xfa\x0b\x8e\xab\xd4\xd7\x8e\x8f\xf5\n!&\xfd\xbb'
hex_ciphertext = 69b0611b958f16d6ed31c6fca5abd5467c98fa0b8eabd4d78e8ff50a2126fdbb
padded_plaintext = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'
hex_padded_plaintext = 6120736563726574206d65737361676510101010101010101010101010101010
plaintext = b'a secret message'
hex_plaintext = 6120736563726574206d657373616765
```

### Task 1.3.2 Checking AES 256 ECB PKCS7 

Test your code now with the message <code>other_message</code>, with a length in char not multiple of 16.

In [54]:
ct = aes256_ecb_pkcs7_encrypt(other_message, key)
print("ct =", ct)

pt = aes256_ecb_pkcs7_decrypt(ct, key)
print("pt =", pt)

ct = b'i\xb0a\x1b\x95\x8f\x16\xd6\xed1\xc6\xfc\xa5\xab\xd5F|\x98\xfa\x0b\x8e\xab\xd4\xd7\x8e\x8f\xf5\n!&\xfd\xbb'
pt = b'a secret message'


You should obtain the following results:
```
other_message = b'This message is important'
padded_other_message = b'This message is important\x07\x07\x07\x07\x07\x07\x07' 
length =  32
ciphertext = b'\xb8{\xe1\x8e\x9fc\xfb\xaa\x13y\xb6\xf5VQW\xc4\xa1\xd0\xc1! .\xff\x9a\xb4\xc9X\x01\xc2\xc9\xf9\xd0'
hex_ciphertext = b87be18e9f63fbaa1379b6f5565157c4a1d0c121202eff9ab4c95801c2c9f9d0
padded_plaintext = b'This message is important\x07\x07\x07\x07\x07\x07\x07'
hex_padded_plaintext = 54686973206d65737361676520697320696d706f7274616e7407070707070707
plaintext = b'This message is important'
hex_plaintext = 54686973206d65737361676520697320696d706f7274616e74
```

### Task 1.3.3 Checking AES 256 ECB PKCS7 against CyberChef

Test again your AES256-ECB-PKCS7 function but this time check your results using the web page tool [CyberChef](https://gchq.github.io/CyberChef/). You have to select, in the left sidebar, 'AES Encrypt' from 'Encryption / Encoding' and ECB mode.

### Task 1.3.4 Generating cryptographically secure random bytes

Till now you have used fixed and given key and iv in your AES256 code. However, ideally, you should randomly generate these parameters each time you need them, at least in the first encryption step. Notice that it is not the same case in the decryption step, as here you must use the same parameters used in the encryption step if you want to correctly recover the plaintext.

Let’s learn first how to generate <ins>cryptographically secure</ins> random bytes.

Read the information provided in the [web page](https://cryptography.io/en/latest/random-numbers/#random-number-generation)

Two possibilities are highlighted:

- Using the function [<code>urandom()</code>](https://docs.python.org/3/library/os.html#os.urandom) from the `os` module 

- Using the function [<code>token_bytes()</code>](https://docs.python.org/3/library/secrets.html#secrets.token_bytes) from the `secrets` module

You should have imported both in Section 0.

Try both ways with the code shown next. Notice that each time you execute the code, new bytes are obtained.

In [55]:
# import os 
# import secrets

print(os.urandom(16))            # generating 16 random bytes
print(secrets.token_bytes(32))   # generating 32 random bytes

b'$\xe2\xf5\xd2\xbe\xce\x8fR\xe4\x19+v\x82\x84T\xd3'
b'\xa9\xcd\xcb\xec\xef\xea\xac\xb6\xabSr\x03\xffBS\xe5\xb67\xb5O\xd3k\xf7%\xfc\xeb\xda\tFuf\x14'


### Task 1.3.5 No-key AES 256 ECB PKCS7 and No-key-no-iv AES 256 CBC PKCS7 encryption functions

Let’s change the your AES256-ECB-PKCS7 encryption code so if no key is provided, the key is randomly generated. Both the ciphertext and the key should be provided as output. Use the code provided below 14 as guide to write a new function <code>aes256_ecb_pkcs7_encrypt_rnd()</code> that incorporates this functionality. 

Note that the function expects two parameters but by writing <code>key=None</code> in the function header for the second one, we allow its absence (that is, it is optional to call the function with a second parameter) and at the same time we provide a default value if it is not provided (in this case this default value is the <code>None</code> value). This means also that, if the key value is not provided, we have to provide a key value inside the function code (in the provided code <code>os.urandom()</code> is used to generate this value randomly). As the key value is needed for decryption, notice that we also return it together with the resulting ciphertext. 

The same happens with the initialization vector <code>iv</code> in the case of the CBC mode. Notice that in the case of AES cipher, the <code>iv</code> size is the same as the block size, that is, 16 bytes.

Re-write the encryption functions as explained taking the following headers and code as starting point.

In [64]:
def aes256_ecb_pkcs7_encrypt_rnd(data_to_encrypt, key=None):
    if key is None:
        key = os.urandom(32) 

    ct = aes256_ecb_pkcs7_encrypt(data_to_encrypt, key)
        
    return ct, key

def aes256_cbc_pkcs7_encrypt_rnd(data_to_encrypt, key=None, iv=None):
    if key is None:
        key = os.urandom(32)
    if iv is None:
        iv = os.urandom(16)
        
    ct = aes256_cbc_pkcs7_encrypt(data_to_encrypt, key, iv)
        
    return ct, key, iv

Once you have the code running, try to encrypt and decrypt the same message and check that different keys and ciphertext are obtained.

In [65]:
# CHECK ECB FUNCTION

print("# ECB FIRST RUN")
ciphertext1, randomkey1 = aes256_ecb_pkcs7_encrypt_rnd(b"This message is important")
print("hex_randomkey1 =", randomkey1.hex(), "\nhex_ciphertext1 =", ciphertext1.hex())
plaintext1 = aes256_ecb_pkcs7_decrypt(ciphertext1, randomkey1)
print("plaintext1 = ", plaintext1)

print("\n# ECB SECOND RUN")
ciphertext2, randomkey2 = aes256_ecb_pkcs7_encrypt_rnd(b"This message is important")
print("hex_randomkey2 =", randomkey2.hex(), "\nhex_ciphertext2 =", ciphertext2.hex())
plaintext2 = aes256_ecb_pkcs7_decrypt(ciphertext2, randomkey2)
print("plaintext2 = ", plaintext2)

# ECB FIRST RUN
hex_randomkey1 = 740e32ffbbee945043509e4f9e79620ee0e4ec8343b4822d3f745951b98192e4 
hex_ciphertext1 = a0fe007127b0e89582b75552818120ecd55dbd7c8b4629a3540c4a6312627a0b
plaintext1 =  b'This message is important'

# ECB SECOND RUN
hex_randomkey2 = 388552279fdc66b07f596cba1aaf2a2ca33fa0b274aee050eaa98f50f901cf5e 
hex_ciphertext2 = f910e71d4b27b0874e27264e6e5681a0d958fbdb36b02fd1bb9413bd052a5e7d
plaintext2 =  b'This message is important'


In [66]:
# CHECK CBC FUNCTION

print("# CBC FIRST RUN")
ciphertext1, randomkey1, randomiv1= aes256_cbc_pkcs7_encrypt_rnd(b"This message is important")
print("hex_randomkey1 =", randomkey1.hex(), "\nhex_ciphertext1 =", ciphertext1.hex(), "\nhex_randomiv1 =", randomiv1.hex())
plaintext1 = aes256_cbc_pkcs7_decrypt(ciphertext1, randomkey1, randomiv1)
print("plaintext1 = ", plaintext1)

print("\n# CBC SECOND RUN")
ciphertext2, randomkey2, randomiv2 = aes256_cbc_pkcs7_encrypt_rnd(b"This message is important")
print("hex_randomkey2 =", randomkey2.hex(), "\nhex_ciphertext2 =", ciphertext2.hex(), "\nhex_randomiv2 =", randomiv2.hex())
plaintext2 = aes256_cbc_pkcs7_decrypt(ciphertext2, randomkey2, randomiv2)
print("plaintext2 = ", plaintext2)

# CBC FIRST RUN
hex_randomkey1 = 082194e1fd21208c6612202b3f38d2ea6f1aeb729925cbe42505520bad07ced1 
hex_ciphertext1 = 433a773465f43e57743de5fe59339b0931d6f298dfd53a568bdb067399474d1e 
hex_randomiv1 = 52fd96ee5387595072139c954dcad097
plaintext1 =  b'This message is important'

# CBC SECOND RUN
hex_randomkey2 = 73f3fc0e9cb4ec9df4218dc33cd531becf8f3bd2fa85514615517c1088eea19e 
hex_ciphertext2 = 1cc92577b7a67f60722dcd1121954fe0ee21a60e2f34ffd1ff8c45a878f6a35a 
hex_randomiv2 = d410d2a64277f8ea162d81cc23c4c70a
plaintext2 =  b'This message is important'


## 1.4 AES256 - CTR MODE

Let’s now use the CTR operation mode. You can consult the library documentation for this mode [here](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.modes.CTR).
Notice that in this case you do not need an initialization vector, but you need a nonce to initialize the counter. 

### Task 1.4.1 Encrypt/decrypt with AES 256 CTR

Review your notes on CTR operation mode.  

**Q:** Does CTR operation mode need padding? That is, is a requirement that the length of the data to encrypt in bytes is multiple of the cipher block size?

**A:** No.

Write two new encryption and one decryption functions based on your AES256-CBC code but that makes use of the CTR mode instead of the CBC one. Notice that PKCS7 padding will NOT be applied.

In [75]:
nonce = b'\xf4$\x89\xab{\xa4-`:\xdeq\xb5m\x1b\xd3\x98'  # 16B

def aes256_ctr_encrypt(data_to_encrypt, key, nonce):
    
    cipher = Cipher(algorithms.AES256(key), modes.CTR(nonce))
    encryptor = cipher.encryptor()
    ct = encryptor.update(data_to_encrypt) + encryptor.finalize()
    
    return ct


def aes256_ctr_decrypt(encrypted_data, key, nonce):
    
    cipher = Cipher(algorithms.AES256(key), modes.CTR(nonce))
    decryptor = cipher.decryptor()
    pt = decryptor.update(encrypted_data) + decryptor.finalize()
    
    return pt


def aes256_ctr_encrypt_rnd(data_to_encrypt, key=None, nonce=None):
    BLOCK_SIZE = 16  # B
    if key is None: key = os.urandom(BLOCK_SIZE)
    if nonce is None: nonce = os.urandom(BLOCK_SIZE)

    ct = aes256_ctr_encrypt(data_to_encrypt, key, nonce)
    
    return ct, key, nonce

In [77]:
# CHECK YOUR CODE

nonce = b'\xf4$\x89\xab{\xa4-`:\xdeq\xb5m\x1b\xd3\x98'

print("other_message :", other_message)
print("key =", key)
print("hex_key =", key.hex())
print("nonce =", nonce)
print("hex_nonce =", nonce.hex())

ciphertext1 = aes256_ctr_encrypt(other_message, key, nonce)
ciphertext2 = aes256_ctr_encrypt(other_message + other_message, key, nonce)

print("ciphertext1 =", ciphertext1.hex(), "\nlength ciphertext1 =", len(ciphertext1))
print("ciphertext2 =", ciphertext2.hex(), "\nlength ciphertext2 =", len(ciphertext2))

plaintext1 = aes256_ctr_decrypt(ciphertext1, key, nonce)
plaintext2 = aes256_ctr_decrypt(ciphertext2, key, nonce)

print("plaintext1 =", plaintext1)
print("plaintext2 =", plaintext2)

other_message : b'This message is important'
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
hex_key = 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a
nonce = b'\xf4$\x89\xab{\xa4-`:\xdeq\xb5m\x1b\xd3\x98'
hex_nonce = f42489ab7ba42d603ade71b56d1bd398
ciphertext1 = a79a1e04ca64996bb6a2644692ec77449d189e30177d2992d8 
length ciphertext1 = 25
ciphertext2 = a79a1e04ca64996bb6a2644692ec77449d189e30177d2992d8e6e3ceba38e8690d5be581bbd93f69a7236edf40d9d900ae96 
length ciphertext2 = 50
plaintext1 = b'This message is important'
plaintext2 = b'This message is importantThis message is important'


You should obtain the following results if you encrypt <code>other_message</code> and two times <code>other_message</code> with the same <code>key</code> previously used and the provided <code>nonce</code>.
```
other_message : b'This message is important'
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
hex_key = 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a
nonce = b'\xf4$\x89\xab{\xa4-`:\xdeq\xb5m\x1b\xd3\x98'
hex_nonce = f42489ab7ba42d603ade71b56d1bd398
ciphertext1 = a79a1e04ca64996bb6a2644692ec77449d189e30177d2992d8 
length ciphertext1 = 25
ciphertext2 = a79a1e04ca64996bb6a2644692ec77449d189e30177d2992d8e6e3ceba38e8690d5be581bbd93f69a7236edf40d9d900ae96 
length  ciphertext2 = 50
plaintext1 = b'This message is important'
plaintext2 = b'This message is importantThis message is important'
```

**Q:** Which length in bytes has the input data to encrypt in Task 1.4.1? And the output ciphertext? Is the length of the output ciphertext multiple of the cipher block size or not and why?

**A:** No.

### Task 1.4.2 Checking AES 256 CTR against CyberChef

Test again your AES256-CTR code but this time check your results using the web page tool [CyberChef](https://gchq.github.io/CyberChef/).

## 1.5 ADDITIONAL ASPECTS OF OPERATION MODES

### Task 1.5.1 No-padding AES 256 ECB and CBC functions

You have already been questioned about the difference in the output lengths when different operation modes are used and when padding is added or not. Now you’ll study what happens when you intentionally modify one bit (or octet) of a ciphertext and then decrypt the modified ciphertext. 

In order to play with modifications in the ciphertext, it is better to avoid PKCS7 padding, as if modifications affect the padding, it will probably raise an error and you’ll not be able to study the effects of the modifications in the plaintext after decryption.  

After all, padding is not _explicitely required_ in these modes, if the size of the message is a multiple of the block size.

Write a new set of functions for AES256-CBC and AES256-ECB encryption and decryption that do not apply padding.

In [80]:
def aes256_cbc_encrypt(data_to_encrypt, key, iv):

    cipher = Cipher(algorithms.AES256(key), modes.CBC(iv))
    encryptor = cipher.encryptor()
    ct = encryptor.update(data_to_encrypt) + encryptor.finalize()

    return ct
    

def aes256_cbc_decrypt(encrypted_data, key, iv):
    
    cipher = Cipher(algorithms.AES256(key), modes.CBC(iv))
    decryptor = cipher.decryptor()
    pt = decryptor.update(encrypted_data) + decryptor.finalize()

    return pt

def aes256_ecb_encrypt(data_to_encrypt, key):
    
    cipher = Cipher(algorithms.AES256(key), modes.ECB())
    encryptor = cipher.encryptor()
    ct = encryptor.update(data_to_encrypt) + encryptor.finalize()

    return ct

def aes256_ecb_decrypt(encrypted_data, key):
    
    cipher = Cipher(algorithms.AES256(key), modes.ECB())
    decryptor = cipher.decryptor()
    pt = decryptor.update(encrypted_data) + decryptor.finalize()

    return pt

Test your functions by encrypting messages with length in chars multiple of 16 and then decrypting the resulting ciphertext.

In [79]:
# CHECKING YOUR CODE

print("message :", message)
print("key :", key)
print("hex_key :", key.hex())
print("iv :", iv)
print("hex_iv :", iv.hex())

print("AES256-CBC ciphertext: ", aes256_cbc_encrypt(message, key, iv))
print("AES256-CBC plaintext: ", aes256_cbc_decrypt(aes256_cbc_encrypt(message, key, iv), key, iv))

print("AES256-ECB ciphertext: ", aes256_ecb_encrypt(message, key))
print("AES256-ECB plaintext: ", aes256_ecb_decrypt(aes256_ecb_encrypt(message, key), key))

message : b'a secret message'
key : b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
hex_key : 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a
iv : b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'
hex_iv : 7fb357841a394b4e7eaf73fc9fb6ccb8
AES256-CBC ciphertext:  b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeN'
AES256-CBC plaintext:  b'a secret message'
AES256-ECB ciphertext:  b'i\xb0a\x1b\x95\x8f\x16\xd6\xed1\xc6\xfc\xa5\xab\xd5F'
AES256-ECB plaintext:  b'a secret message'


With the previously used values for <code>message</code>, <code>key</code> and <code>iv</code>, you should obtain the following results:
```
message : b'a secret message'
key : b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
hex_key : 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a
iv : b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'
hex_iv : 7fb357841a394b4e7eaf73fc9fb6ccb8
AES256-CBC ciphertext:  b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeN'
AES256-CBC plaintext:  b'a secret message'
AES256-ECB ciphertext:  b'i\xb0a\x1b\x95\x8f\x16\xd6\xed1\xc6\xfc\xa5\xab\xd5F'
AES256-ECB plaintext:  b'a secret message'
```

### Task 1.5.2 Effects of modifying ciphertext under different operation modes

Now, analyse how the decrypted plaintext changes after modifying one bit of the ciphertext at different positions (one in the first block, another in the second block…).

Suggested steps:
- Select a message with at least 48-char length (remember that the length in char has to be multiple of 16)
- Select a key, and iv and nonce if necessary
- Encrypt the message operating the cipher with the selected mode
- Save the ciphertext in one variable (`ciphertext`)
- Modify the previous ciphertext in one bit and save the result in another variable (modified_ciphertext)
- Decrypt both the ciphertext and its modified version, saving the decryptions, respectively, in two variables: `plaintext` and `modified_plaintext`
- Analyse how the ciphertext modification has affected the plaintext

In [127]:
# THIS SHIT DOESN'T WORK FOR SOME FUCKING REASON

KEY_SIZE = 32
BLOCK_SIZE = 16

key = os.urandom(KEY_SIZE)
iv = os.urandom(BLOCK_SIZE)
nonce = os.urandom(BLOCK_SIZE)
message = b'a secret message'

print("key =", key)
print("iv =", iv)
print("nonce =", nonce)
print("message =", message)

# encrypt
ct_cbc = aes256_cbc_encrypt(message, key, iv)
ct_ecb = aes256_ecb_encrypt(message, key)
ct_ctr = aes256_ctr_encrypt(message, key, nonce)

print("")
print("ct_cbc =", ct_cbc)
print("ct_ecb =", ct_ecb)
print("ct_ctr =", ct_ctr)

# add 1

mod_ct_cbc = (int.from_bytes(ct_cbc, "big") + 1).to_bytes(len(ct_cbc), "big")
mod_ct_ecb = (int.from_bytes(ct_ecb, "big") + 1).to_bytes(len(ct_ecb), "big")
mod_ct_ctr = (int.from_bytes(ct_ctr, "big") + 1).to_bytes(len(ct_ctr), "big")


print("")
print("mod_ct_cbc =", mod_ct_cbc)
print("mod_ct_ecb =", mod_ct_ecb)
print("mod_ct_ctr =", mod_ct_ctr)

# decrypt

mod_pt_cbc = aes256_cbc_decrypt(mod_ct_cbc, key, iv)
mod_pt_ecb = aes256_ecb_decrypt(mod_ct_cbc, key)
mod_pt_ctr = aes256_ctr_decrypt(mod_ct_cbc, key, nonce)

print("")
print("mod_pt_cbc =", mod_pt_cbc)
print("mod_pt_ecb =", mod_pt_ecb)
print("mod_pt_ctr =", mod_pt_ctr)

key = b'\x1e\xac\x1a\x1dK#\x8c\x0ebp7`\x0c\xe0\xf7\xf2\xad\x98K\x08}\x16LiZ\xe1.j\xbc\xc2\xc9Z'
iv = b'0\xf0\x99\xcem\xef\xbb\x98:\xf8x<<\x91ae'
nonce = b'\xd6\xb2N\xf7\xa7S\n\xd6C\x02\xff\xd2T\xfef\xdc'
message = b'a secret message'

ct_cbc = b'e\xecO\x92\x98\x155\xca\x1c\xc6`I9\xee\x86\xad'
ct_ecb = b'\xc4\x9a#\xf6G#\xcf\xec+\xc2\xf4&\xfd%\xba\xd3'
ct_ctr = b'\xb7\x18\x10&\xd2\xa5\xed\xf5\xba-,W\xda9\xcf)'

mod_ct_cbc = b'e\xecO\x92\x98\x155\xca\x1c\xc6`I9\xee\x86\xae'
mod_ct_ecb = b'\xc4\x9a#\xf6G#\xcf\xec+\xc2\xf4&\xfd%\xba\xd4'
mod_ct_ctr = b'\xb7\x18\x10&\xd2\xa5\xed\xf5\xba-,W\xda9\xcf*'

mod_pt_cbc = b'v\xa5\xf5z@\xect\xd3!\x18\x8b5^\xd4p\x95'
mod_pt_ecb = b'FUl\xb4-\x03\xcfK\x1b\xe0\xf3\tbE\x11\xf0'
mod_pt_ctr = b'\xb3\xd4,\xd1)\xc2\xbdK\x86\x86)m\x90\xb6.\xe2'


How many bits (or bytes) are affected in each case (ECB, CBC, CTR)? Which are their positions? Can you explain the reasons in each case considering the course notes on operation modes?  

- **CBC and ECB modes:** The entire block of the altered byte is corrupted.
- **CTR mode:** Only the affected byte is corrupted.

## 1.6 OTHER CIPHERS AND KEY SIZES

### Task 1.6.1 Camellia symmetric block cipher

Now, create new functions similar to the ones already coded but that in this case use the Camellia cipher and other key sizes. Consult the library documentation to know more about the Camellia cipher and available key sizes for both AES and Camellia.

In [136]:
message = b"a secret message"
key = os.urandom(KEY_SIZE)

cipher = Cipher(algorithms.Camellia(key), mode = modes.ECB())

encryptor = cipher.encryptor()
ct = encryptor.update(message) + encryptor.finalize()

print("ct =", ct)

decryptor = cipher.decryptor()
pt = decryptor.update(ct) + decryptor.finalize()

print("pt =", pt)

ct = b')\x9ct=.\xaf-\xcfW]M\x059F\x95O'
pt = b'a secret message'


## 1.7 EXCHANGING SECRET MESSAGES WITH A PARTNER

### Task 1.7.1 Exchanging secret messages

Choose a partner among your classmates and exchange with him/her secret messages. You’ll need to send the key and other needed parameters along with the ciphertext. The receiver will also need to know the used algorithm, operation mode and key/block size if there are available more than one.

**Notice that in real life keys should NEVER be sent UNPROTECTED.**

# 2. Symmetric Stream Ciphers

Next, you’ll learn how to encrypt a simple text with symmetric stream ciphers. First, you’ll use the CHACHA20 cipher and then you’ll use the `pylfsr` module to generate pseudorandom key streams using LFSRs.

## 2.1 CHACHA20 STREAM CIPHER

### Task 2.1.1 CHACHA20 encryption/decryption functions

Now read the information and take the code provided as example in the following [web page](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.algorithms.ChaCha20)

Take the code in the example and use it as base to create the functions whose headers are defined next. Test your functions with the data and results of Code Snippet `29` and Code Snippet `30`. 

In [137]:
def chacha20_encrypt(message, key, nonce):
    
    cipher = Cipher(algorithms.ChaCha20(key, nonce), mode = None)
    encryptor = cipher.encryptor()
    ct = encryptor.update(message) + encryptor.finalize()

    return ct

def chacha20_encrypt_rnd(message, key=None, nonce=None):
    if key is None:
        key = os.urandom(32)    # key must have 256 bits
    if nonce is None:
       nonce = os.urandom(16)   # nonce must have 128 bits
    
    ct = chacha20_encrypt(message, key, nonce)
    
    return ct, key, nonce;

def chacha20_decrypt(ciphertext, key, nonce):
    
    cipher = Cipher(algorithms.ChaCha20(key, nonce), mode = None)
    encryptor = cipher.encryptor()
    pt = encryptor.update(ciphertext) + encryptor.finalize()

    return pt

In [139]:
# Check your code

print("message :", message)
print("other_message :", other_message)
print("key =", key)
print("hex_key =", key.hex())
print("nonce =", nonce)
print("hex_nonce =", nonce.hex())

print("")

print("ciphertext chacha20 message=", chacha20_encrypt(message, key, nonce))
print("hex_ciphertext chacha20 message=", chacha20_encrypt(message, key, nonce).hex())

print("plaintext chacha20 message=", chacha20_decrypt(chacha20_encrypt(message, key, nonce), key, nonce))
print("hex_plaintext chacha20 message=", chacha20_decrypt(chacha20_encrypt(message, key, nonce), key, nonce).hex())

print("ciphertext chacha20 other_message=", chacha20_encrypt(other_message, key, nonce))
print("hex_ciphertext chacha20 other_message=", chacha20_encrypt(other_message, key, nonce).hex())

print("plaintext chacha20 other_message=", chacha20_decrypt(chacha20_encrypt(other_message, key, nonce), key, nonce))
print("hex_plaintext chacha20 other_message=", chacha20_decrypt(chacha20_encrypt(other_message, key, nonce), key, nonce).hex())


message : b'a secret message'
other_message : b'This message is important'
key = b'\xdd\x15S\x12\x83\r}\x95\xfa4\xd6\ti\xcfh\xf0\x98\xedm\xe3\xbe\xd2s\xf8\xb5\xed\x1f+\x88\xb2\x85\xaa'
hex_key = dd155312830d7d95fa34d60969cf68f098ed6de3bed273f8b5ed1f2b88b285aa
nonce = b'\xd6\xb2N\xf7\xa7S\n\xd6C\x02\xff\xd2T\xfef\xdc'
hex_nonce = d6b24ef7a7530ad64302ffd254fe66dc

ciphertext chacha20 message= b'\x85o\x1f^1\xaf\xa46\xcfy\x8e\xe9P!Y\xb3'
hex_ciphertext chacha20 message= 856f1f5e31afa436cf798ee9502159b3
plaintext chacha20 message= b'a secret message'
hex_plaintext chacha20 message= 6120736563726574206d657373616765
ciphertext chacha20 other_message= b"\xb0'\x05Hr\xb0\xa41\x9cu\x8c\xff\x03)M\xf6\xfd^_\xfc\x80x \xc8e"
hex_ciphertext chacha20 other_message= b027054872b0a4319c758cff03294df6fd5e5ffc807820c865
plaintext chacha20 other_message= b'This message is important'
hex_plaintext chacha20 other_message= 54686973206d65737361676520697320696d706f7274616e74


You should obtain the following result:
```
message : b'a secret message'
other_message : b'This message is important'
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
hex_key = 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a
nonce = b'\xf4$\x89\xab{\xa4-`:\xdeq\xb5m\x1b\xd3\x98'

hex_nonce = f42489ab7ba42d603ade71b56d1bd398
ciphertext chacha20 message= b'Q\x02\x96\x92\xed\xc1\x90-\x97r\xaa/.\xb3`\xfd'
hex_ciphertext chacha20 message= 51029692edc1902d9772aa2f2eb360fd
plaintext chacha20 message= b'a secret message'
hex_plaintext chacha20 message= 6120736563726574206d657373616765
ciphertext chacha20 other_message= b'dJ\x8c\x84\xae\xde\x90*\xc4~\xa89}\xbbt\xb8\x19G\xa7!\xac7\xed8z'
hex_ciphertext chacha20 other_message= 644a8c84aede902ac47ea8397dbb74b81947a721ac37ed387a
plaintext chacha20 other_message= b'This message is important'
hex_plaintext chacha20 other_message= 54686973206d65737361676520697320696d706f7274616e74  
```

## 2.2 GENERATING KEYSTREAMS WITH LFSRS

Now you’ll make use of the [`pylfsr`](https://lfsr.readthedocs.io/en/latest/index.html) module. This library generates pseudorandom bit streams using, among other generators, LFSRs. It requires to have installed the `numpy` library, and in the case that you are using pylfsr’s graphical functions, the `matplotlib` library. 

### Task 2.2.1 Generating a key stream

Have a look at the information presented in [this web page](https://lfsr.readthedocs.io/en/latest/index.html).  
Consider the example [“Example 2: 5-bit LFSR with custum state and feedback polynomial”](https://lfsr.readthedocs.io/en/latest/LFSR_Examples.html#example-2-5-bit-lfsr-with-custum-state-and-feedback-polynomial).

Using the code in the example, generate 10 bits using primitive polynomial $x^5 + x^4 + x^3+ x^2 + 1$

In [142]:
# from pylfsr import LFSR

state = [0, 0, 0, 1, 0]   # seed = S1 S2 S3 S4 S5 
fpoly = [5, 4, 3, 2]         # polynomial C5=1 C4=1 C3=1 C2=1 C1=0
L = LFSR(fpoly=fpoly, initstate=state, verbose=True)  # LFSR initialization
L.info()                  # prints information of the configured LFSR
tempseq = L.runKCycle(10) # generation of 10 bits 

5-bit LFSR with feedback polynomial  x^5 + x^4 + x^3 + x^2 + 1 with
Expected Period (if polynomial is primitive) =  31
Computing configuration is set to Fibonacci with output sequence taken from 5-th (-1) register
Current :
 State        :  [0 0 0 1 0]
 Count        :  0
 Output bit   :  -1
 feedback bit :  -1
S:  [0 0 0 1 0]
S:  [1 0 0 0 1]
S:  [1 1 0 0 0]
S:  [1 1 1 0 0]
S:  [0 1 1 1 0]
S:  [1 0 1 1 1]
S:  [1 1 0 1 1]
S:  [1 1 1 0 1]
S:  [1 1 1 1 0]
S:  [1 1 1 1 1]


The generated sequence is returned to the variable <code>tempseq</code>. Print its value and check that you obtain the same result as shown next: <code> [0 1 0 0 0 1 1 1 0 1] </code>

In [143]:
print(tempseq)

[0 1 0 0 0 1 1 1 0 1]


### Task 2.2.2 Checking that you generate the same keystream

Make sure you understand how the class works by checking by hand that the code obtains the same 10-bit sequence that you compute. Use the same parameters:
- poly(x) = $x^5 + x^4 + x^3+ x^2 + 1 $
- seed = [S1 S2 S3 S4 S5] = [0, 0, 0, 1, 0]